# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [19]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [21]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [24]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [27]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [28]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [41]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [43]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    27| 2.5976338|
|     31|   1.0|    13|  2.113986|
|     31|   1.0|     5| 1.3165921|
|     31|   2.0|    25|0.16370271|
|     85|   1.0|    28|-2.5285664|
|     85|   1.0|    26|0.37620115|
|     85|   1.0|    12| 0.8253538|
|     85|   3.0|     1| 1.6069186|
|     85|   1.0|    13| 2.2720711|
|     85|   5.0|    16|0.80576146|
|     85|   1.0|    15|0.54832166|
|     85|   1.0|     4|  3.144216|
|     65|   1.0|    28|  -2.03051|
|     65|   2.0|     3|  3.801642|
|     65|   1.0|     2| 1.7128268|
|     53|   3.0|    13| 3.4453833|
|     53|   1.0|     6| 1.8362958|
|     53|   1.0|     9| 1.8954519|
|     78|   1.0|    22| 0.5302301|
|     78|   1.0|    13| 0.5055496|
+-------+------+------+----------+
only showing top 20 rows



In [29]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.751143638387403


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [47]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [48]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     13|    11|
|     18|    11|
|     30|    11|
|     66|    11|
|     70|    11|
|     75|    11|
|     78|    11|
|     79|    11|
|     99|    11|
+-------+------+



In [49]:
reccomendations = model.transform(single_user)

In [54]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     30|    11|  5.578189|
|     13|    11|  3.257565|
|     70|    11| 2.7580981|
|     99|    11| 1.7420897|
|     18|    11| 1.5150304|
|     75|    11|   1.34218|
|     79|    11| 0.9733073|
|     66|    11| 0.5732717|
|     78|    11| 0.4434041|
|      0|    11|  -1.85298|
+-------+------+----------+



# Great Job!